## Extracting Data:
 For programmes cutoffs and subject requirements

In [1]:
#importing libraries
import pdfplumber
import re
import csv
import pandas as pd
from collections import OrderedDict


### Extracting Data for subject requirements

In [4]:
def normalize_mat_alternative(text):
    if not text:
        return text

    # Normalize combined variants first
    text = re.sub(
        r'\bMAT\s+ALTERNATIVE\s*(A\s*[/|]\s*B|B\s*[/|]\s*A|A\s+or\s+B|B\s+or\s+A)\b',
        'MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B',
        text,
        flags=re.IGNORECASE
    )
    
    # Then normalize individual variants
    text = re.sub(
        r'\bMAT\s+ALTERNATIVE\s*A\b',
        'MAT_ALTERNATIVE_A',
        text,
        flags=re.IGNORECASE
    )
    text = re.sub(
        r'\bMAT\s+ALTERNATIVE\s*B\b',
        'MAT_ALTERNATIVE_B',
        text,
        flags=re.IGNORECASE
    )

    # Normalize GROUPs
    text = re.sub(r'\sGROUP\s', '_GROUP_', text, flags=re.IGNORECASE)

    return text.strip()

def normalize_dashes(text):
    if not text:
        return text
    return re.sub(r'[–—−]', '-', text)

def process_subject(subject):
    if not subject:
        return ""
    return normalize_mat_alternative(subject.strip())

def split_degrees(text):
    if not text:
        return []
    parts = re.split(r'(?=\bBachelor\b)', text)
    return [p.strip() for p in parts if p.strip()]

def extract_subgrade_pairs(text):
    pairs = []
    entries = [entry.strip() for entry in text.split(",")]
    pattern = re.compile(r'^(.+?)[\s\-–—:]+([A-F][+-]?)$', re.IGNORECASE)
    for entry in entries:
        entry = normalize_dashes(entry)
        match = pattern.match(entry)
        if match:
            subject = match.group(1).strip()
            grade = match.group(2).strip().upper()
            pairs.append((subject, grade))
    return pairs

def split_subjects_simple(line):
    tokens = line.split()
    subjects = []
    current = []
    delimiters = {'/', 'or', '|'}

    for tok in tokens:
        if tok.lower() in delimiters:
            current.append('/')
        else:
            if current and current[-1] != '/':
                subjects.append(' '.join(current))
                current = []
            current.append(tok)

    if current:
        subjects.append(' '.join(current))

    return subjects

# === Main Parser ===

def parse_cluster_from_text(file_path):
    extracted_data = []
    current_subcluster = None
    current_degrees = []
    last_subject_block = {}
    current_grade_line = ""

    def save_block():
        if current_subcluster and current_degrees and last_subject_block:
            cluster_num = int(re.match(r'^(\d+)', current_subcluster).group(1))

            subject_grades = {
                "subject_1_grade": "Null",
                "subject_2_grade": "Null",
                "subject_3_grade": "Null",
                "subject_4_grade": "Null"
            }

            normalized_subjects = {
                key: process_subject(val)
                for key, val in last_subject_block.items()
            }

            current_grade_line_norm = normalize_dashes(current_grade_line)

            if current_grade_line_norm:
                pairs = extract_subgrade_pairs(current_grade_line_norm)
                for subj, grade in pairs:
                    norm_subj = process_subject(subj)
                    for key, norm_stored_subj in normalized_subjects.items():
                        if norm_stored_subj.lower() == norm_subj.lower() or norm_stored_subj.lower() in norm_subj.lower():
                            grade_key = key + "_grade"
                            subject_grades[grade_key] = grade

            extracted_data.append({
                "degrees": current_degrees.copy(),
                "cluster": cluster_num,
                "subcluster": current_subcluster,
                "subject_1": normalized_subjects.get("subject_1", ""),
                "subject_2": normalized_subjects.get("subject_2", ""),
                "subject_3": normalized_subjects.get("subject_3", ""),
                "subject_4": normalized_subjects.get("subject_4", ""),
                **subject_grades
            })

    with open(file_path, "r", encoding="utf-8") as f:
        lines = [normalize_mat_alternative(line.strip()) for line in f if line.strip()]

    i = 0
    while i < len(lines):
        line = lines[i]

        # Detect subject header line
        if all(s in line.lower() for s in ["subject 1", "subject 2", "subject 3", "subject 4"]):
           

            subject_lines = []
            j = i + 1
            while j < len(lines) and not re.search(r'\b\d{1,2}[A-Z]\b', lines[j]) and (j - i) <= 1:
                subject_lines.append(lines[j])
                j += 1

            combined_subject_text = " ".join(subject_lines).strip()
            combined_subject_text = re.sub(r'\s+', ' ', combined_subject_text)

            subjects = split_subjects_simple(combined_subject_text)
            subjects = list(OrderedDict.fromkeys([process_subject(s.strip()) for s in subjects if s.strip()]))

            last_subject_block = {
                "subject_1": subjects[0] if len(subjects) > 0 else "",
                "subject_2": subjects[1] if len(subjects) > 1 else "",
                "subject_3": subjects[2] if len(subjects) > 2 else "",
                "subject_4": subjects[3] if len(subjects) > 3 else ""
            }

            i = j
            continue

        # Detect subcluster line (✅ FIX APPLIED HERE)
        subcl_match = re.search(r'\b(\d{1,2}[A-Z])\b(.*)', line)
        if subcl_match:
            save_block()  # Save previous block
            current_subcluster = subcl_match.group(1).upper()
            current_degrees = []
            rest_of_line = subcl_match.group(2).strip()
            rest_of_line = normalize_dashes(rest_of_line)

            degrees_part = ""
            grade_part = rest_of_line

            bachelor_index = rest_of_line.lower().find("bachelor")
            if bachelor_index != -1:
                grade_part = rest_of_line[:bachelor_index].strip()
                degrees_part = rest_of_line[bachelor_index:].strip()
                if degrees_part:
                    current_degrees.extend(split_degrees(degrees_part))
            else:
                grade_part = rest_of_line

            current_grade_line = grade_part
            i += 1
            continue

        # Continuation of degrees on next line
        if current_subcluster and re.match(r'^\s*Bachelor', line, re.IGNORECASE):
            current_degrees.extend(split_degrees(line.strip()))

        i += 1

    save_block()  # Final save at end
    return extracted_data

# === Export to Excel ===

def export_to_excel(data, excel_path):
    rows = []
    for entry in data:
        for degree in entry["degrees"]:
            rows.append({
                "programme_name": degree,
                "cluster": entry["cluster"],
                "subcluster": entry["subcluster"],
                "subject_1": entry["subject_1"],
                "subject_1_grade": entry["subject_1_grade"],
                "subject_2": entry["subject_2"],
                "subject_2_grade": entry["subject_2_grade"],
                "subject_3": entry["subject_3"],
                "subject_3_grade": entry["subject_3_grade"],
                "subject_4": entry["subject_4"],
                "subject_4_grade": entry["subject_4_grade"]
            })
    
    df = pd.DataFrame(rows)
    df.to_excel(excel_path, index=False)

# === Usage === 
text_file = "degree_req_text.txt"  # Input text file
parsed_data = parse_cluster_from_text(text_file)
export_to_excel(parsed_data, "degree_requirements_from_text.xlsx")

# Optional: preview as DataFrame
degree_req_df = pd.DataFrame([
    {
        "programme_name": degree,
        "cluster": entry["cluster"],
        "subcluster": entry["subcluster"],
        "subject_1": entry["subject_1"],
        "subject_1_grade": entry["subject_1_grade"],
        "subject_2": entry["subject_2"],
        "subject_2_grade": entry["subject_2_grade"],
        "subject_3": entry["subject_3"],
        "subject_3_grade": entry["subject_3_grade"],
        "subject_4": entry["subject_4"],
        "subject_4_grade": entry["subject_4_grade"]
    }
    for entry in parsed_data for degree in entry["degrees"]
])

degree_req_df.head()


,programme_name,cluster,subcluster,subject_1,subject_1_grade,subject_2,subject_2_grade,subject_3,subject_3_grade,subject_4,subject_4_grade
0,Bachelor of Laws (LL.B.),1,1A,ENG/KIS,B,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,Null,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV / any_...,Null
1,Bachelor in Business Administration,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
2,Bachelor of Library & Information Science,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
3,"Bachelor in Business Administration, With IT",2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
4,Bachelor of Library and Information Studies,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null


In [5]:
degree_req_df.subcluster.unique()

array(['1A', '2A', '2B', '3A', '3B', '3C', '3D', '3E', '4A', '4B', '5A',
       '5B', '5C', '5D', '5E', '5F', '6A', '6B', '6C', '7A', '7B', '7C',
       '8A', '9A', '9B', '9C', '9D', '10A', '10B', '10C', '11A', '12A',
       '13A', '13B', '13C', '13D', '13E', '13F', '13G', '14A', '15A',
       '15B', '15C', '15D', '15E', '15F', '15G', '16A', '17A', '18A',
       '19A', '19B', '19C', '19D', '19E', '19F', '19G', '19H', '19I',
       '19J', '19K', '20A'], dtype=object)

In [6]:
degree_req_df.head(8)

,programme_name,cluster,subcluster,subject_1,subject_1_grade,subject_2,subject_2_grade,subject_3,subject_3_grade,subject_4,subject_4_grade
0,Bachelor of Laws (LL.B.),1,1A,ENG/KIS,B,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,Null,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV / any_...,Null
1,Bachelor in Business Administration,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
2,Bachelor of Library & Information Science,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
3,"Bachelor in Business Administration, With IT",2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
4,Bachelor of Library and Information Studies,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
5,Bachelor of Business and Office Management,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
6,Bachelor of Science in Information Sciences,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null
7,Bachelor of Business Management,2,2A,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B,C+,Any_GROUP_II / any_GROUP_III,Null,A_GROUP_II / a_GROUP_III / any_GROUP_IV /,Null


In [7]:
for i in range(1,5):
    col = "subject_"+str(i)
    print(col + " unique values")
    col+="_grade"
    print(degree_req_df["subject_"+str(i)+"_grade"].unique())

subject_1 unique values
['B' 'Null' 'C+' 'C']
subject_2 unique values
['Null' 'C+' 'C' 'B']
subject_3 unique values
['Null' 'C' 'C+' 'B']
subject_4 unique values
['Null' 'C']


In [8]:
degree_req_df[(degree_req_df.subject_1_grade=="Null")& (degree_req_df.subject_2_grade=="Null")&(degree_req_df.subject_3_grade=="Null")]

,programme_name,cluster,subcluster,subject_1,subject_1_grade,subject_2,subject_2_grade,subject_3,subject_3_grade,subject_4,subject_4_grade
46,Bachelor of Project Planning and Management,2,2B,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,Any_GROUP_III,Null,A_GROUP_II / 2nd_GROUP_III / any_GROUP_IV / an...,Null
47,Bachelor of Hotel & Hospitality Management,2,2B,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,Any_GROUP_III,Null,A_GROUP_II / 2nd_GROUP_III / any_GROUP_IV / an...,Null
48,Bachelor of Science (Project Planning Management),2,2B,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,Any_GROUP_III,Null,A_GROUP_II / 2nd_GROUP_III / any_GROUP_IV / an...,Null
49,Bachelor of Hotel Management,2,2B,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,Any_GROUP_III,Null,A_GROUP_II / 2nd_GROUP_III / any_GROUP_IV / an...,Null
50,Bachelor of Science in Project Management,2,2B,ENG/KIS,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,Any_GROUP_III,Null,A_GROUP_II / 2nd_GROUP_III / any_GROUP_IV / an...,Null
...,...,...,...,...,...,...,...,...,...,...,...
752,Bachelor of Science in Agricultural Education ...,19,19I,ENG,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,2nd_GROUP_II,Null,KIS / 3rd_GROUP_II / 2nd_GROUP_III / any_GROUP...,Null
753,Bachelor of Science (Agricultural Education an...,19,19I,ENG,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,2nd_GROUP_II,Null,KIS / 3rd_GROUP_II / 2nd_GROUP_III / any_GROUP...,Null
754,Bachelor of Education (Computer Studies),19,19J,ENG,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,2nd_GROUP_II,Null,KIS / 3rd_GROUP_II / 2nd_GROUP_III / any_GROUP...,Null
755,Bachelor of Education (ICT),19,19J,ENG,Null,MAT_ALTERNATIVE_A/MAT_ALTERNATIVE_B / any_GROU...,Null,2nd_GROUP_II,Null,KIS / 3rd_GROUP_II / 2nd_GROUP_III / any_GROUP...,Null


In [9]:
degree_req_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769 entries, 0 to 768
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   programme_name   769 non-null    object
 1   cluster          769 non-null    int64 
 2   subcluster       769 non-null    object
 3   subject_1        769 non-null    object
 4   subject_1_grade  769 non-null    object
 5   subject_2        769 non-null    object
 6   subject_2_grade  769 non-null    object
 7   subject_3        769 non-null    object
 8   subject_3_grade  769 non-null    object
 9   subject_4        769 non-null    object
 10  subject_4_grade  769 non-null    object
dtypes: int64(1), object(10)
memory usage: 66.2+ KB


In [10]:
degree_req_df=degree_req_df[~degree_req_df.programme_name.duplicated()]

In [11]:
degree_req_df.programme_name.drop_duplicates(inplace=True)

C:\Users\Mich\AppData\Local\Temp/ipykernel_17984/277535980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  degree_req_df.programme_name.drop_duplicates(inplace=True)


In [15]:
degree_req_df.to_excel("degree_requirements_from_text.xlsx")

### Extracting Data for programmes cutoffs:
    From 2018 to 2024

In [16]:
def parse_cutoff_pdf(pdf_path):
    data = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue
            lines = [line.strip() for line in text.split('\n') if line.strip()]

            for line in lines:
                # Match lines that start with digits followed by space and text
                m = re.match(r'^(\d+)\s+(.*)', line)
                if not m:
                    continue

                first_num = m.group(1)  # e.g., "11105101"
                rest_line = m.group(2)

                # Serial number is all but last 7 digits of first_num
                if len(first_num) <= 7:
                    # skip lines where first number is too short to have serial + prog code
                    continue
                serial = first_num[:-7]
                prog_code = first_num[-7:]

                # Now rest_line contains institution + programme + cutoffs

                # Split rest_line by space
                parts = rest_line.split()

                # Cutoffs are always last 7 tokens (years 2018-2024)
                cutoffs = parts[-7:]
                if len(cutoffs) < 7:
                    cutoffs += ['-'] * (7 - len(cutoffs))

                # Institution and programme are the rest
                institution_and_programme = parts[:-7]

                # Now we separate institution and programme by looking for keywords
                institution_parts = []
                programme_parts = []

                found_programme_start = False
                for word in institution_and_programme:
                    if not found_programme_start and re.search(r'BACHELOR|DIPLOMA|CERTIFICATE|MASTER|PHD', word, re.I):
                        found_programme_start = True
                    if found_programme_start:
                        programme_parts.append(word)
                    else:
                        institution_parts.append(word)

                institution_name = " ".join(institution_parts)
                programme_name = " ".join(programme_parts)

                data.append({
                    "programme_code": prog_code,
                    "institution": institution_name,
                    "programme_name": programme_name,
                    "cutoff_2018": cutoffs[0],
                    "cutoff_2019": cutoffs[1],
                    "cutoff_2020": cutoffs[2],
                    "cutoff_2021": cutoffs[3],
                    "cutoff_2022": cutoffs[4],
                    "cutoff_2023": cutoffs[5],
                    "cutoff_2024": cutoffs[6],
                })

    return data

def export_to_excel(data, excel_path):
    df = pd.DataFrame(data)
    df.to_excel(excel_path, index=False)
pdf_file = "DEGREE_CUTOFFS_18-24.pdf"
parsed_data = parse_cutoff_pdf(pdf_file)
export_to_excel(parsed_data, "degree_cutoffs.xlsx")
print("Excel file 'degree_cutoffs.xlsx' created successfully.")


Excel file 'degree_cutoffs.xlsx' created successfully.


In [17]:
cutoff_df = pd.read_excel("degree_cutoffs.xlsx")

In [18]:
cutoff_df.head(5)

,programme_code,institution,programme_name,cutoff_2018,cutoff_2019,cutoff_2020,cutoff_2021,cutoff_2022,cutoff_2023,cutoff_2024
0,1105101,CHUKA UNIVERSITY,BACHELOR OF ARTS,25.798,26.729,28.419,25.974,28.566,23.714,24.851
1,1087101,KISII UNIVERSITY,BACHELOR OF ARTS,23.93,24.763,27.318,22.926,26.315,24.286,22.173
2,1073101,RONGO UNIVERSITY,BACHELOR OF ARTS,-,-,-,-,27.808,24.574,24.252
3,1057101,EGERTON UNIVERSITY,BACHELOR OF ARTS,24.851,26.54,28.227,22.926,26.783,24.188,21.799
4,1164101,MULTIMEDIA UNIVERSITY OF KENYA,"BACHELOR OF ARTS (PSYCHOLOGY, SOCIOLOGY & POLI...",-,-,-,24.456,26.207,22.449,22.851


In [19]:
cutoff_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   programme_code  2050 non-null   int64 
 1   institution     2050 non-null   object
 2   programme_name  2046 non-null   object
 3   cutoff_2018     2050 non-null   object
 4   cutoff_2019     2050 non-null   object
 5   cutoff_2020     2050 non-null   object
 6   cutoff_2021     2050 non-null   object
 7   cutoff_2022     2050 non-null   object
 8   cutoff_2023     2050 non-null   object
 9   cutoff_2024     2050 non-null   object
dtypes: int64(1), object(9)
memory usage: 160.3+ KB
